In [2]:
import numpy as np
import pandas as pd
from faker import Faker
from scipy.stats import skewnorm

# Severity distribution according to GEMA 5.0 guidelines
GEMA_SEVERITY_DIST = {
    "Intermittent": 0.25,
    "Mild Persistent": 0.35,
    "Moderate Persistent": 0.25,
    "Severe Persistent": 0.15
}

# Adherence parameters from REALISE study
SYMBICORT_ADHERENCE_PARAMS = {
    "Intermittent": (1.8, 2.2),
    "Mild Persistent": (2.5, 2.5),
    "Moderate Persistent": (3.2, 1.8),
    "Severe Persistent": (3.5, 1.2)
}

# District configuration with geographic and behavioral parameters
DISTRICT_CONFIG = {
    'Sant Marti': {
        'center': (41.3984, 2.1901),
        'work_prob': 0.6,
        'entertainment_prob': 0.3,
        'radius': 0.015
    },
    'Sants-Montjuic': {
        'center': (41.3601, 2.1331),
        'work_prob': 0.4,
        'entertainment_prob': 0.4,
        'radius': 0.02
    },
    'Eixample': {
        'center': (41.3954, 2.1611),
        'work_prob': 0.7,
        'entertainment_prob': 0.6,
        'radius': 0.01
    },
    'Gracia': {
        'center': (41.4024, 2.1553),
        'work_prob': 0.3,
        'entertainment_prob': 0.7,
        'radius': 0.008
    },
    'Ciutat Vella': {
        'center': (41.3805, 2.1733),
        'work_prob': 0.5,
        'entertainment_prob': 0.8,
        'radius': 0.006
    },
    'Horta-Guinardo': {
        'center': (41.4302, 2.1600),
        'work_prob': 0.2,
        'entertainment_prob': 0.2,
        'radius': 0.025
    },
    'Les Corts': {
        'center': (41.3853, 2.1315),
        'work_prob': 0.5,
        'entertainment_prob': 0.4,
        'radius': 0.018
    }
}

PERIOD = 24 * 365  # One year of hourly data

def generate_realistic_location(home_district, work_district, num_hours):
    """Generate realistic daily movement patterns with stable locations"""
    home = DISTRICT_CONFIG[home_district]
    work = DISTRICT_CONFIG[work_district]
    locations = []
    
    current_location = (home['center'][0], home['center'][1])
    
    for hour in range(num_hours):
        day_type = 'weekday' if (hour // 24) % 7 < 5 else 'weekend'
        hour_of_day = hour % 24
        
        # Nighttime sleep pattern (10pm-6am)
        if 22 <= hour_of_day or hour_of_day < 6:
            new_location = (
                np.random.normal(home['center'][0], home['radius']/1000),
                np.random.normal(home['center'][1], home['radius']/1000)
            )
        
        # Weekday pattern
        elif day_type == 'weekday':
            if 6 <= hour_of_day < 9:  # Morning commute
                new_location = (
                    np.random.normal((home['center'][0] + work['center'][0])/2, 0.003),
                    np.random.normal((home['center'][1] + work['center'][1])/2, 0.003)
                )
            elif 9 <= hour_of_day < 18:  # Workday
                new_location = (
                    np.random.normal(work['center'][0], work['radius']/1000),
                    np.random.normal(work['center'][1], work['radius']/1000)
                )
            elif 18 <= hour_of_day < 20:  # Evening commute
                new_location = (
                    np.random.normal((home['center'][0] + work['center'][0])/2, 0.003),
                    np.random.normal((home['center'][1] + work['center'][1])/2, 0.003)
                )
            else:  # Evening leisure (8pm-10pm)
                if hour_of_day % 2 == 0:  # Stay in same leisure location for 2h
                    leisure_district = np.random.choice(
                        ['Gracia', 'Eixample', home_district],
                        p=[0.4, 0.4, 0.2]
                    )
                    current_leisure = DISTRICT_CONFIG[leisure_district]['center']
                
                new_location = (
                    np.random.normal(current_leisure[0], 0.002),
                    np.random.normal(current_leisure[1], 0.002)
                )
        
        # Weekend pattern
        else:
            if 10 <= hour_of_day < 18:  # Daytime outing
                if hour_of_day == 10:  # Choose weekend location once per day
                    weekend_district = np.random.choice(
                        [home_district, 'Ciutat Vella', 'Sants-Montjuic'],
                        p=[0.4, 0.4, 0.2]
                    )
                    current_weekend = DISTRICT_CONFIG[weekend_district]['center']
                
                new_location = (
                    np.random.normal(current_weekend[0], 0.003),
                    np.random.normal(current_weekend[1], 0.003)
                )
            else:  # Home evenings
                new_location = (
                    np.random.normal(home['center'][0], home['radius']/1000),
                    np.random.normal(home['center'][1], home['radius']/1000)
                )
        
        locations.append(new_location)
    
    # Unzip coordinates
    lats, lons = zip(*locations)
    return list(lats), list(lons)

def generate_gema_cohort(num_patients=1000):
    fake = Faker('es_ES')
    np.random.seed(7)
    
    # Generate realistic age distribution
    ages = np.clip(
        skewnorm.rvs(4, loc=35, scale=15, size=num_patients).astype(int),
        12,  # Minimum diagnosis age
        90   # Realistic maximum age
    )
    
    # District distribution based on Barcelona population
    districts = list(DISTRICT_CONFIG.keys())
    district_probs = [0.18, 0.15, 0.22, 0.12, 0.10, 0.13, 0.10]
    
    patients = pd.DataFrame({
        "patient_id": [f"PAT-{i:04d}" for i in range(num_patients)],
        "gender": np.random.choice(["M", "F"], num_patients, p=[0.45, 0.55]),
        "age": ages,
        "home_district": np.random.choice(districts, num_patients, p=district_probs),
        "gema_severity": np.random.choice(
            list(GEMA_SEVERITY_DIST.keys()),
            num_patients,
            p=list(GEMA_SEVERITY_DIST.values())
        )
    })
    
    # Assign work districts with normalized probabilities
    patients["work_district"] = patients["home_district"].apply(
        lambda hd: np.random.choice(
            [hd] + [d for d in districts if d != hd],
            p=np.append(
                DISTRICT_CONFIG[hd]['work_prob'],
                [(1 - DISTRICT_CONFIG[hd]['work_prob']) / (len(districts) - 1)] * (len(districts) - 1)
            ).tolist()
        )
    )
    
    # Generate adherence values
    def _get_adherence(severity_group):
        severity = severity_group.name
        a, b = SYMBICORT_ADHERENCE_PARAMS[severity]
        return np.clip(np.random.beta(a, b, size=len(severity_group)), 0.3, 0.95)
    
    patients["symbicort_adherence"] = patients.groupby("gema_severity")["gema_severity"].transform(_get_adherence)
    
    # Calculate base exacerbation risk
    exacerbation_risk = patients["gema_severity"].map({
        "Intermittent": 0.12,
        "Mild Persistent": 0.27,
        "Moderate Persistent": 0.43,
        "Severe Persistent": 0.68
    })
    patients["base_exacerbation_risk"] = exacerbation_risk * (1.15 - patients["symbicort_adherence"])
    
    # Generate comorbidities
    patients["has_allergic_rhinitis"] = np.where(
        patients["gema_severity"].isin(["Moderate Persistent", "Severe Persistent"]),
        np.random.binomial(1, 0.75, num_patients),
        np.random.binomial(1, 0.45, num_patients)
    )
    
    patients["has_COPD"] = np.where(
        patients["age"] > 40,
        np.random.binomial(1, np.clip(0.015 * (patients["age"] - 40), 0, 0.35)),
        0
    )
    
    # Generate inhaler events with realistic locations
    inhaler_dfs = []
    for severity, group in patients.groupby("gema_severity"):
        n_patients = len(group)
        base_puffs = {
            "Intermittent": 0.25,
            "Mild Persistent": 0.55,
            "Moderate Persistent": 1.35,
            "Severe Persistent": 2.75
        }[severity]
        
        # Generate circadian rhythm pattern
        circadian = np.sin(2 * np.pi * np.arange(PERIOD) / 24 - np.pi/2) * 0.35 + 1
        circadian = np.clip(circadian, 0.4, 1.6)
        
        # Generate puffs with Poisson distribution
        puffs = np.random.poisson(
            lam=(base_puffs * circadian.reshape(1, -1) * 
                 group["symbicort_adherence"].values.reshape(-1, 1)),
            size=(n_patients, PERIOD)
        )
        
        # Generate realistic locations for each patient
        location_data = []
        for _, patient in group.iterrows():
            lats, lons = generate_realistic_location(
                patient['home_district'],
                patient['work_district'],
                PERIOD
            )
            location_data.append(pd.DataFrame({
                'latitude': lats,
                'longitude': lons
            }))
        
        location_df = pd.concat(location_data, ignore_index=True)
        
        # Build final DataFrame for this severity group
        severity_df = pd.DataFrame({
            "patient_id": np.repeat(group["patient_id"], PERIOD),
            "timestamp": np.tile(pd.date_range("2024-01-01", periods=PERIOD, freq='h'), n_patients),
            "puffs": puffs.flatten(),
            "latitude": location_df['latitude'].values,
            "longitude": location_df['longitude'].values,
            "device_type": "Symbicort Turbuhaler"
        })
        
        inhaler_dfs.append(severity_df)
    
    return patients, pd.concat(inhaler_dfs, ignore_index=True)

# Generate and save data
patients, inhaler_events = generate_gema_cohort(1000)

# Add date partition column
inhaler_events["date"] = inhaler_events["timestamp"].dt.date

# Add district column based on coordinates
districts_list = list(DISTRICT_CONFIG.keys())
centers_array = np.array([[info['center'][0], info['center'][1]] for info in DISTRICT_CONFIG.values()])

lats = inhaler_events['latitude'].to_numpy()
lons = inhaler_events['longitude'].to_numpy()

# Calculate closest districts using vectorized operations
diff_lat = lats[:, np.newaxis] - centers_array[:, 0]
diff_lon = lons[:, np.newaxis] - centers_array[:, 1]
distances = diff_lat**2 + diff_lon**2
closest_indices = np.argmin(distances, axis=1)

inhaler_events['district'] = [districts_list[i] for i in closest_indices]

In [3]:
# Save to Parquet with partitioning
inhaler_events.to_parquet(
    "../data/raw/iot_inhaler/inhaler_events.parquet",
    partition_cols=["date"],
    compression='snappy'
)

patients.to_parquet(
    "../data/raw/iot_inhaler/patients.parquet",
    compression='snappy'
)

In [4]:
def clinical_validation(patients_df):
    # 1. Define strict categorical order
    gema_order = [
        "Intermittent", 
        "Mild Persistent", 
        "Moderate Persistent", 
        "Severe Persistent"
    ]
    
    patients_df["gema_severity"] = pd.Categorical(
        patients_df["gema_severity"],
        categories=gema_order,
        ordered=True
    )
    
    # 2. Get distribution while maintaining original order
    severity_counts = patients_df["gema_severity"].value_counts(normalize=True, sort=False)
    
    # 3. Ordered comparison with adjusted tolerance
    expected_dist = np.array(list(GEMA_SEVERITY_DIST.values()))
    obtained_dist = severity_counts.values
    
    assert np.allclose(
        obtained_dist,
        expected_dist,
        atol=0.035,  # Tolerance for N=1000
        rtol=0.05    # Additional relative tolerance
    ), f"""Distribution out of expected range:
    Expected: {dict(zip(gema_order, expected_dist))}
    Actual: {dict(zip(gema_order, obtained_dist))}"""
    
    # Remaining validations...
    print("✓ Validation successful")

clinical_validation(patients)

✓ Validation successful


In [5]:
import folium
import pandas as pd

df_patients = inhaler_events[inhaler_events['patient_id'] == 'PAT-0004'][:(24*10)]

# Create base map centered on Barcelona
barcelona_coords = [41.3851, 2.1734]
patient_map = folium.Map(location=barcelona_coords, zoom_start=13)

# Add markers for each patient
for idx, row in df_patients.iterrows():
    popup_text = f"""
    Patient ID: {row['patient_id']}<br>
    Time: {row['timestamp']}<br>
    Puffs: {row['puffs']}<br>
    District: {row['district']}
    """
    puffs = row['puffs']
    color = 'green' if puffs < 1 else 'blue' if puffs <= 2 else 'orange' if puffs <= 5 else 'red'
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=250),
        icon=folium.Icon(color=color, icon='user-md', prefix='fa')
    ).add_to(patient_map)

patient_map